# This notebook generates binary representation of input matrices in a form suitable for input stationary systolic array where the weights are transformed to trapezoid for data orchestration 

In [1]:
import numpy as np
mat_H_A = 16 # A is west
mat_W_A = 4
mat_H_B = 4 # B is north
mat_W_B = 1024
# A = np.zeros([mat_H, mat_W], dtype=int)
# B = np.zeros([mat_W, mat_H], dtype=int)
# for i in range(mat_H):
#     for j in range(mat_W):
#         A[i][j]=i*mat_W+j
# for i in range(mat_W):
#     for j in range(mat_H):
#         B[i][j]=i*mat_H+j
A = np.random.randint(low=0, high=10000, size=(mat_H_A, mat_W_A), dtype=np.int64)
B = np.random.randint(low=0, high=10000, size=(mat_H_B, mat_W_B), dtype=np.int64)
print(A)
print(B)
# B_transposed = B.transpose()
print(np.dot(A,B))
B_original = B.copy()
A_original = A.copy()

[[9063 7471 4559 7712]
 [2955 9800 8309 9249]
 [6839 4952 6478 7090]
 [9707 4353 7527 9366]
 [6946 3201 8858 5787]
 [6520 9188 9104 9140]
 [6197 7532 7458  534]
 [4794 7802 1776 8818]
 [9373 5897 1239 6988]
 [6674 1451 6888 3152]
 [1284 4897 8059 2308]
 [6766 7939 5712 4533]
 [4628 4491 8503 3514]
 [2171 6136 5271 9485]
 [8148 5979 9088 2129]
 [7414 9718 6499 6738]]
[[9922 8647 1723 ... 9062 7535 5023]
 [5948 7990 3296 ... 2295 9615 1482]
 [ 411 3119 5010 ... 5048 6483 7547]
 [6636 6463 4983 ... 2101 8216 1160]]
[[187411175 202123228 101509451 ... 138491595 233041159  99948164]
 [152401273 189545943 125108122 ... 110645191 246349956 102803428]
 [147022752 164730865  95889639 ... 120936892 199393659  98805027]
 ...
 [123146431 145541081  97636354 ...  80291715 187456778  70781322]
 [134270760 160333165  89885475 ... 137908234 195292633 120845058]
 [178748829 205573753 110940294 ... 136451968 246795485 108506631]]


In [2]:

# from fxpmath import Fxp
# word_size  = 32
# frac_size = 0
# bin_ex = Fxp(61, True, word_size, frac_size).bin()
# bin_ex


from fxpmath import Fxp
word_size  = 32
frac_size = 0
#Sorting full matrix
# Accepts 2D weight matrix (NORTH) and reorders rows to get minmum switching possible
# Returns new matrix with low switching between rows along with a vector of new indices
def sortFullMatrix_BNorth(mat1):
    height = np.size(mat1, 0)
    width = np.size(mat1, 1)
#     switchingact = np.zeros(height)
    original_index=[]
    original_index = np.asanyarray([i for i in range(height)])
    for row in range(height - 2): # Iterate until total rows - 2 or 3
#         print('working on row no :', row)
        sw_vector = np.zeros(height)
        for ii in range(row+1, height): # Iterate until total rows - 2 or 3
#             print('comparing switching between row :', row, 'and row', ii)
            temp0=0
            for col in range(width):
                row0 = Fxp(mat1[row][col], True, word_size, frac_size)
                row1 = Fxp(mat1[ii ][col], True, word_size, frac_size)
                temp0 = temp0 + (row0 ^ row1).bin().count('1')
            sw_vector[ii] =  temp0
        if(row==0):
            switchingact = sw_vector
#         print('Switching vector', sw_vector)
#         new_order = np.argsort(sw_vector)
        new_row = np.argmin(sw_vector[row + 1:]) + row + 1  # Adding 1 to cancel the offset from removing (row + 1)th index
#         print('matrix before sorting')
#         print(row+1)
#         print(new_row)
        # switch row i+1 and row i+2

        mat1[[row+1, new_row]] = mat1[[new_row, row+1]]
        original_index[[row+1, new_row]] = original_index[[new_row, row+1]]
#         print('original_index after being sorted:', original_index)
#         print('matrix after sorting')
#         print(mat1)
    return mat1, original_index, switchingact # Returning original_index to rearrange activation maps
B_trans_sorted, original_index, _ = sortFullMatrix_BNorth(B.transpose())
print(B_original.transpose())
print(B_trans_sorted)
# A_rearranged = A[original_index, :] # No need to rearrange A as the sorting of B is done along its filter dimension
# print(original_index)
# print(A_rearranged)


[[9922 5948  411 6636]
 [8647 7990 3119 6463]
 [1723 3296 5010 4983]
 ...
 [9062 2295 5048 2101]
 [7535 9615 6483 8216]
 [5023 1482 7547 1160]]
[[9922 5948  411 6636]
 [5874 3385 5114 8684]
 [5844 1596 8939 9180]
 ...
 [ 659 9446  804 9810]
 [ 461 8857 4238 8927]
 [8511 6543 1334 8076]]


In [3]:


def calc_sw_act(mat1):
    height = np.size(mat1, 0)
    width = np.size(mat1, 1)
    sw_vector = np.zeros(height-1)
    for row in range(height - 1): # Iterate until total rows - 1
            temp0=0
            for col in range(width):
                row0 = Fxp(mat1[row][col], True, word_size, frac_size)
                row1 = Fxp(mat1[row + 1 ][col], True, word_size, frac_size)
                temp0 = temp0 + (row0 ^ row1).bin().count('1')
            sw_vector[row] =  temp0
    return sw_vector # Returning original_index to rearrange activation maps
B_trans = B_original.transpose()
sw_vector = calc_sw_act(B_trans)  
print('mat before sorting', sw_vector)
print(np.sum(sw_vector))
sw_vector = calc_sw_act(B_trans_sorted) #As B_trans_sorted is already transposed
print('mat after sorting', sw_vector)
print(np.sum(sw_vector))

##### No need to rearrange A as the sorting of B is done along its filter dimension
# sw_vector = calc_sw_act(A_original)
# print('mat before sorting', sw_vector)
# print(np.sum(sw_vector))
# sw_vector = calc_sw_act(A_rearranged)
# print('mat after sorting', sw_vector)
# print(np.sum(sw_vector))

# Transforming matrices to trapezoid
# For input stationary systolic array weight matrix is from west is not fed in trapezoid feed as the matrix from north
# Wait for done signal until all weight matrix elements get saved in its corresponding cell
# Start feeding matrix from north in trapezoid sequence


mat before sorting [20. 32. 29. ... 27. 29. 18.]
27736.0
mat after sorting [17. 16. 16. ... 25. 28. 30.]
16706.0


In [4]:

# Transforming matrices to trapezoid
B_trans_trap = np.zeros([mat_H_B+mat_W_B, mat_H_B], dtype=int)
B_trans_sorted_trap = np.zeros([mat_H_B+mat_W_B, mat_H_B], dtype=int)
###########################
# Add zero column from right of a numpy array A (West)
# zeros_column = np.zeros((mat_H_A, 1))
# A_in0 = np.hstack((zeros_column, A))
# print(A_in0)
# A_rearranged_in0 = np.hstack((zeros_column, A_rearranged))
# print(A_rearranged_in0)
############################
for i in range(mat_W_B): 
    for j in range(mat_H_B):
        B_trans_trap[mat_H_B+mat_W_B-j-i-1][j]=B_trans[i][j]
print(B_trans_trap)

for i in range(mat_W_B):
    for j in range(mat_H_B):
        B_trans_sorted_trap[mat_H_B+mat_W_B-j-i-1][j]=B_trans_sorted[i][j]
print(B_trans_sorted_trap)


[[   0    0    0    0]
 [   0    0    0 1160]
 [   0    0 7547 8216]
 ...
 [1723 7990  411    0]
 [8647 5948    0    0]
 [9922    0    0    0]]
[[   0    0    0    0]
 [   0    0    0 8076]
 [   0    0 1334 8927]
 ...
 [5844 3385  411    0]
 [5874 5948    0    0]
 [9922    0    0    0]]


In [5]:

# Transforming trapezoid to its hex representation and save it in text/mem file
### No need to rearrange A as the sorting of B is done along its filter dimension
######################################## Mat A (West in)
with open('./mem_files_InputStationary/inp_west_'+str(mat_H_A)+'x'+str(mat_W_A)+'.mem', 'w') as f:
    for j in range(0,mat_W_A):
        for i in range(0,mat_H_A):
            f.write('{0:x}'.format(int(A[i][mat_W_A-1-j])))
            f.write('\n')
# with open('./mem_files_InputStationary/inp_west_rearranged_'+str(mat_H_A)+'x'+str(mat_W_A)+'.mem', 'w') as f:
#     for j in range(0,mat_W_A+1):
#         for i in range(0,mat_H_A):
#             f.write('{0:x}'.format(int(A[i][mat_W_A-j])))
#             f.write('\n')
########################################## Mat B (North in)
with open('./mem_files_InputStationary/inp_north_'+str(mat_W_B)+'x'+str(mat_H_B)+'.mem', 'w') as f:
    for i in range(0,mat_H_B+mat_W_B):
        for j in range(0,mat_H_B):
            f.write('{0:x}'.format(int(B_trans_trap[mat_H_B+mat_W_B-i-1][j])))
            f.write('\n')
with open('./mem_files_InputStationary/inp_north_sorted_'+str(mat_W_B)+'x'+str(mat_H_B)+'.mem', 'w') as f:
    for i in range(0,mat_H_B+mat_W_B):
        for j in range(0,mat_H_B):
            f.write('{0:x}'.format(int(B_trans_sorted_trap[mat_H_B+mat_W_B-i-1][j])))
            f.write('\n')



In [9]:
xx = np.dot(A_original,B_original)
print(xx[15,:])

[178748829 205573753 110940294 ... 136451968 246795485 108506631]


In [12]:
xx[15,:]

array([178748829, 205573753, 110940294, ..., 136451968, 246795485,
       108506631], dtype=int64)

In [16]:
x=177183061
x in xx[15, :]

False